# **Libraries & Cordinates**

In [11]:
!pip -q install bokeh==3.4.1
from math import pi, log, tan

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, Select
from bokeh.layouts import column
from bokeh.tile_providers import get_provider, Vendors

output_notebook()

# Helpers: WGS84 (lon, lat) → Web Mercator (x, y) for tile maps
def wgs84_to_web_mercator(lon, lat):
    k = 6378137.0
    x = lon * (pi/180.0) * k
    y = log(tan((pi/4.0) + (lat*pi/360.0))) * k
    return x, y

# **Preparing Location Data**

In [12]:
# Data (edit me!)
# Minimal sample: name, lon, lat, category, rating
places = [
    {"name":"Gateway of India","lon":72.8347,"lat":18.9220,"category":"Landmark","rating":4.7},
    {"name":"Chhatrapati Shivaji Terminus","lon":72.8365,"lat":18.9402,"category":"Landmark","rating":4.6},
    {"name":"Sanjay Gandhi National Park","lon":72.9106,"lat":19.2147,"category":"Park","rating":4.5},
    {"name":"Jio World Drive","lon":72.8566,"lat":19.0609,"category":"Mall","rating":4.4},
    {"name":"Marine Drive","lon":72.8236,"lat":18.9432,"category":"Promenade","rating":4.8},
    {"name":"Taj Mahal Palace","lon":72.8337,"lat":18.9217,"category":"Hotel","rating":4.7},
    {"name":"Prithvi Theatre","lon":72.8296,"lat":19.1163,"category":"Culture","rating":4.6},
]

# Convert coordinates to Web Mercator for Bokeh tile maps
for p in places:
    p["x"], p["y"] = wgs84_to_web_mercator(p["lon"], p["lat"])

# Build ColumnDataSource (Bokeh's reactive table)
source_all = ColumnDataSource({k: [d[k] for d in places] for k in places[0].keys()})
source = ColumnDataSource(source_all.data.copy())  # filtered view

# Get unique categories for the dropdown
categories = sorted(list({p["category"] for p in places}))
categories = ["All"] + categories

# Compute a nice initial map range (center & zoom)
xs, ys = source_all.data["x"], source_all.data["y"]
xmid = sum(xs)/len(xs)
ymid = sum(ys)/len(ys)
span = max(max(xs)-min(xs), max(ys)-min(ys)) * 1.2 or 1000  # fallback
x_range = (xmid - span/2, xmid + span/2)
y_range = (ymid - span/2, ymid + span/2)

# **Interactive City Map**

In [14]:
# Figure + tiles + points + hover
tile_provider = get_provider(Vendors.OSM)
#tile_provider = get_provider(Vendors.ESRI_IMAGERY)


p = figure(x_range=x_range, y_range=y_range,
           x_axis_type="mercator", y_axis_type="mercator",
           width=800, height=500,
           title="Your City Explorer — filter by category")

p.add_tile(tile_provider)

# Plot points
r = p.circle(x="x", y="y", size=12, source=source, alpha=0.9)

# Hover shows details
p.add_tools(HoverTool(tooltips=[
    ("Name", "@name"),
    ("Category", "@category"),
    ("Rating", "@rating"),
    ("Lon,Lat", "@lon, @lat")
], renderers=[r]))

# Dropdown to filter
select = Select(title="Category", value="All", options=categories)

# JS callback to update the filtered source without a Python server
callback = CustomJS(args=dict(src_all=source_all, src=source, select=select), code="""
    const cat = select.value;
    const data_all = src_all.data;
    const data = src.data;
    const n = data_all['name'].length;

    // Reset arrays
    for (const k in data) { data[k] = []; }

    for (let i=0; i<n; i++) {
        if (cat === "All" || data_all['category'][i] === cat) {
            for (const k in data_all) {
                data[k].push(data_all[k][i]);
            }
        }
    }
    src.change.emit();
""")

select.js_on_change("value", callback)

layout = column(select, p)
show(layout)
